В этой статье, мы углубимся в изучение Jupyter Interactive Widgets и Ipyleaflet package, для визуализации картографических данных в Jupyter notebook.

Мы будем использовать интересные данные из [Единый государственный реестр объектов культурного наследия(памятников истории и культуры) народов РФ](https://opendata.mkrf.ru/opendata/7705851331-egrkn/)

In [1]:
import zipfile, json
import pandas as pd
from ipyleaflet import AwesomeIcon, Map, Marker, MarkerCluster, Heatmap, WidgetControl, basemaps, basemap_to_tiles
from ipywidgets import Dropdown

Загружаем данные в виде zip архива [ссылка](https://opendata.mkrf.ru/opendata/7705851331-egrkn/)

In [ ]:
mkrf_path = './data/data-49-structure-6.csv.zip'
zf = zipfile.ZipFile(mkrf_path) 
df = pd.read_csv(zf.open('data-49-structure-6.csv'))

In [ ]:
fdf = df[['Объект', 'Регион', 'Вид объекта', 'Категория историко-культурного значения', 'На карте']].copy()
fdf.head(2)

In [ ]:
fdf = fdf[fdf['Регион'] == 'г. Санкт-Петербург'].dropna()

In [ ]:
fdf.to_pickle("./data/spb-mkrf.pkl")

In [2]:
spbdf = pd.read_pickle("./data/spb-mkrf.pkl")

In [3]:
rtypes = spbdf['Вид объекта'].unique()
rtypes

array(['Памятник', 'Ансамбль', 'Достопримечательное место'], dtype=object)

In [4]:
spbdf['Категория историко-культурного значения'].unique()

array(['Федерального значения', 'Регионального значения'], dtype=object)

In [5]:
icon_monument = AwesomeIcon(name='monument',marker_color='orange')
icon_group = AwesomeIcon(name='object-group',marker_color='green')
icon_landmark = AwesomeIcon(name='landmark',marker_color='red')
markers = {e:[] for e in list(rtypes)}
for r in spbdf.itertuples():
    lng, lat = json.loads(r[5])['coordinates']
    rtype = r[3]
    if rtype == 'Памятник': 
        icon = icon_monument
    elif rtype == 'Ансамбль':
        icon = icon_group
    elif rtype == 'Достопримечательное место':
        icon = icon_landmark
    markers[rtype].append(Marker(location=[lat, lng], icon=icon, title=str(r[1])))

Эта карта использует Leaflet Marker Cluster plugin для отображения памятников на карте Санкт-Петербурга от Esri.

Виджет выпадающего списка Ipywidgets Dropdown для просмотра объектов разных видов, заменяющий слои карты.

In [6]:
center = [59.928871952236804, 30.322255639340106]
zoom = 12
m = Map(center=center, zoom=zoom, basemap=basemaps.Esri.WorldStreetMap)

In [7]:
marker_cluster = MarkerCluster(markers=markers['Памятник'])
m.add_layer(marker_cluster)
# Adds a Dropdown widget
dropdown = Dropdown(
    options=list(markers.keys()),
    value='Памятник',
    description='Категории'
)

# Handles Dropdown control event
def on_click(change):
    type_name = change['new']
    global marker_cluster
    m.remove_layer(marker_cluster)
    marker_cluster = MarkerCluster(markers=markers[type_name])
    m.add_layer(marker_cluster)
    
dropdown.observe(on_click, 'value')

# Adds control to the map
basemap_control = WidgetControl(widget=dropdown, position='topright')
m.add_control(basemap_control)

m

Map(center=[59.928871952236804, 30.322255639340106], controls=(ZoomControl(options=['position', 'zoom_in_text'…

![rec](images/marker-clusters.gif)

Эта карта использует Leaflet heatmap plugin для отображения зон города Санкт-Петербурга максимально насыщенных объектами культурного наследия. На этот раз используем основу Esri.WorldTopoMap

In [8]:
center = [59.928871952236804, 30.322255639340106]
zoom = 12
mh = Map(center=center, zoom=zoom, basemap=basemaps.Esri.WorldTopoMap)

In [9]:
def create_heatmap_data():
    "Return a list of some random lat/lng/value triples."
    data = []
    for r in spbdf.itertuples():
        lng, lat = json.loads(r[5])['coordinates']
        data.append([lat, lng, 1])
    return data

In [10]:
heat = Heatmap(locations=create_heatmap_data(), 
               radius=20, blur=10, 
               gradient = {0.4: 'red', 
                           0.6: 'yellow', 
                           0.7: 'lime', 
                           0.8: 'cyan', 
                           1.0: 'blue'})
mh.add_layer(heat)

mh

Map(center=[59.928871952236804, 30.322255639340106], controls=(ZoomControl(options=['position', 'zoom_in_text'…

![rec](images/heatmap.gif)